## CML Bees - Simplifying SageMaker Object Detection 

### Running 1 - Prepare Data

#### 1_prepare_data/docker/code/utils/tf_record_util.py

In [23]:
import os
import io
import json
import jsonlines
import random
import logging
#from utils import dataset_util
from TensorFlow.models.research.object_detection.utils import dataset_util
from PIL import Image
import tensorflow as tf

In [56]:
class TfRecordGenerator:
    def __init__(self, image_dir, manifest, label_map, output_dir):
        self.image_dir = image_dir
        self.manifest = manifest
        self.label_map = label_map
        self.output_dir = output_dir

    def generate_tf_records(self):
        with jsonlines.open(self.manifest, 'r') as reader:
            ground_truth_annotations = list(reader)
            dataset = split_dataset(ground_truth_annotations)
            for subset in dataset:
                logging.info(f'GENERATING TF RECORD FOR {subset}')
                writer = tf.io.TFRecordWriter(os.path.join(self.output_dir, f'{subset}.records'))
                for image_annotations in dataset[subset]:
                    annotation_dict = json.loads(json.dumps(image_annotations))
                    tf_example = self._create_tf_example(annotation_dict['source-ref'],
                                                         annotation_dict['bees-500']['annotations'])
                    writer.write(tf_example.SerializeToString())
                writer.close()

    def _create_tf_example(self, s3_image_path, image_dir, annotations):
        image_name = os.path.basename(s3_image_path)
        image_path = f'{self.image_dir}/{image_name}'
        im = Image.open(image_path)

        # READ IMAGE FILE
        with tf.io.gfile.GFile(image_path, 'rb') as fid:
            encoded_jpg = fid.read()

        encoded_jpg_io = io.BytesIO(encoded_jpg)
        encoded_jpg_io.seek(0)
        image = Image.open(encoded_jpg_io)
        image_width, image_height = image.size
        if image.format != 'JPEG':
            image = image.convert('RGB')

        xmins = []
        ymins = []
        xmaxs = []
        ymaxs = []
        classes = []
        classes_text = []
        for a in annotations:
            x = a['left']
            y = a['top']
            width = a['width']
            height = a['height']
            class_id = a['class_id']
            xmins.append(float(x) / image_width)
            xmaxs.append(float(x + width) / image_width)
            ymins.append(float(y) / image_height)
            ymaxs.append(float(y + height) / image_height)
            class_name = self.label_map[int(class_id)]
            classes_text.append(class_name.encode('utf8'))
            classes.append(class_id)

        feature_dict = {
            'image/height': dataset_util.int64_feature(image_height),
            'image/width': dataset_util.int64_feature(image_width),
            'image/filename': dataset_util.bytes_feature(bytes(image_name, 'utf-8')),
            'image/source_id': dataset_util.bytes_feature(bytes(image_name.replace('.jpg', ''), 'utf-8')),
            'image/encoded': dataset_util.bytes_feature(encoded_jpg),
            'image/format': dataset_util.bytes_feature('jpeg'.encode('utf8')),
            'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
            'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
            'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
            'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
            'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
            'image/object/class/label': dataset_util.int64_list_feature(classes),
        }
        example = tf.train.Example(features=tf.train.Features(feature=feature_dict))
        return example

In [57]:
def split_dataset(list_images):
    dataset = {}
    random.seed(42)
    random.shuffle(list_images)
    num_train = int(0.9 * len(list_images))
    dataset['train'] = list_images[:num_train]
    dataset['validation'] = list_images[num_train:]
    logging.info(f'TRAINING EXAMPLES: %d - VALIDATION EXAMPLES: %d', len(dataset['train']), len(dataset['validation']))
    return dataset

#### 1_prepare_data/docker/code/prepare_data.py

In [58]:
import json
from tf_record_util_copy import TfRecordGenerator

In [59]:
input_folder = '/home/cdsw/data/data'
ground_truth_manifest = '/home/cdsw/data/data/output.manifest'
label_map = '{"0": "bee"}' # class mapping here - e.g. - each class ID should map to the human readable equivalent
output_folder = '/home/cdsw/data/output'

In [60]:
def prepare_data(input_folder, ground_truth_manifest, label_map, output_folder):
    
    label_map = json.loads(args.label_map)

    # Feed in necessary path variables from above operations
    tf_record_generator = TfRecordGenerator(image_dir=input_folder,
                                            manifest=ground_truth_manifest,
                                            label_map=label_map,
                                            output_dir=output_folder)

    print('GENERATING TF RECORD FILES')
    tf_record_generator.generate_tf_records()

    print('GENERATING LABEL MAP FILE')
    with open(f'{output_folder}/label_map.pbtxt', 'w') as label_map_file:
        for item in label_map:
            label_map_file.write('item {\n')
            label_map_file.write(' id: ' + str(int(item) + 1) + '\n')
            label_map_file.write(" name: '" + label_map[item] + "'\n")
            label_map_file.write('}\n\n')

    print('FINISHED')

In [61]:
label_map = json.loads(label_map)

In [62]:
# Feed in necessary path variables from above operations
tf_record_generator = TfRecordGenerator(image_dir=input_folder,
                                        manifest=ground_truth_manifest,
                                        label_map=label_map,
                                        output_dir=output_folder)

In [63]:
print('GENERATING TF RECORD FILES')
tf_record_generator.generate_tf_records()

GENERATING TF RECORD FILES


In [65]:
print('GENERATING LABEL MAP FILE')
with open(f'{output_folder}/label_map.pbtxt', 'w') as label_map_file:
    for item in label_map:
        label_map_file.write('item {\n')
        label_map_file.write(' id: ' + str(int(item) + 1) + '\n')
        label_map_file.write(" name: '" + label_map[item] + "'\n")
        label_map_file.write('}\n\n')

GENERATING LABEL MAP FILE


### Train model 2 - Prepare Data